NEW CONTINUING: after [finding errors](https://github.com/Data-Science-for-Linguists-2019/Animated-Movie-Gendered-Dialogue/blob/master/code/Disney_code/Messy_Disney_Movies.ipynb) in Tangled and Frozen, I attempt using BS4.

## An Attempt at Web Scraping

In [1]:
from bs4 import BeautifulSoup

In [2]:
import re

In [3]:
import requests

In [4]:
link = "https://imeldasanders123.wordpress.com/disney-tangled-script/"

In [5]:
page_response = requests.get(link, timeout=5)

In [6]:
page_response

<Response [200]>

In [7]:
page_content = BeautifulSoup(page_response.content, "html.parser")

In [8]:
#print(page_content)

In [9]:
type(page_content)

bs4.BeautifulSoup

In [10]:
#page_content.body.div id

In [11]:
new_html = page_content.find("div", {"class": "entry-content"}) #this is the script portion!

In [12]:
#new_html

In [13]:
#This gets rid of all the embedded videos in the middle of the script
for span in new_html.find_all('span'):
    span.replace_with('')

In [14]:
#new_html

I'm really not familiar with webscraping or beautiful soup objects... but I know how to work with strings, so I'm gonna turn this into one.

In [15]:
tangled_string = str(new_html)

In [16]:
tangled_string[:500]

'<div class="entry-content">\n<p>—Key—<br/>\nR:Rapunzel, Mandy Moore<br/>\nF:Eugene Fitzherbert/Flynn Rider, Zachary Levi<br/>\nG:Mother Gothel, Donna Murphy<br/>\nHook:Hook-Hand Thug, Brad Garrett<br/>\nBig:Big Nose Thug, Jeffrey Tambor<br/>\nSt:The Stabbington Brothers, Ron Perlman (only the speaking one; the\xa0one without the eye-patch)<br/>\nVlad:Vladamir, Richard Kiel<br/>\nShorty:Shorty Thug, Paul F. Tompkins<br/>\nYoung R:Young Rapunzel, Delaney Rose Stein<br/>\nCapt:Captain of the Guard, M. C. Gainey<'

In [17]:
tangled_string[-500:]

'wrapper jetpack-likes-widget-unloaded" data-name="like-post-frame-14362632-3055-5ca7fadaecec7" data-src="//widgets.wp.com/likes/index.html?ver=20190321#blog_id=14362632&amp;post_id=3055&amp;origin=imeldasanders123.wordpress.com&amp;obj_id=14362632-3055-5ca7fadaecec7" id="like-post-wrapper-14362632-3055-5ca7fadaecec7"><h3 class="sd-title">Like this:</h3><div class="likes-widget-placeholder post-likes-widget-placeholder" style="height: 55px;"> </div><a class="sd-link-color"></a></div></div> </div>'

In [18]:
splitting = tangled_string.split("BEGIN")

In [19]:
rough_script = splitting[1] #gets rid of jargon before script

In [20]:
new_split = rough_script.split("THE END")

In [21]:
script = new_split[0] #gets ride of jargon after script

In [22]:
script[:500]

'—<br/>\nF: This is the story of how I died.<br/>\nDon’t worry, this is actually a very fun story and the truth is, it\xa0isn’t even mine.<br/>\nThis is the story of a girl named Rapunzel and it starts with the\xa0sun.</p>\n<p>Now, once upon a time, a single drop of sunlight fell from the\xa0heavens and from this small drop of sun, grew a magic, golden flower. It\xa0had the\xa0ability to heal the sick and injured.<br/>\nOh, you see that old woman over there? You might want to remember<br/>\nher. She’s kind of importa'

In [23]:
script = script.replace('\xa0', ' ')

In [24]:
script = script.replace('&amp', '')

In [25]:
script = re.sub(r"\n", '', script)

In [26]:
action = re.findall(r"\[[^\]]*\]", script)

In [27]:
len(action)

297

In [28]:
for bracket in action[:10]:
    print(bracket)

[gasps at cut hair]
[Pascal flees window to hide]
[Pascal sniggers. R finds it]
[Pascal frowns]
[Pascal motions to go outside]
[Pascal sticks out tongue]
[leaves window]
[paints lights]
[back to window]
[St lower F to steal crown. A guard sneezes]


In [29]:
script = re.sub(r"\[[^\]]*\]", '', script)

In [30]:
script_test = re.sub(r"(<br/>|<p>|</p>)(([A-Z][a-z]* )?[A-Z][a-z]*:)", r"_NEWLINE_\2_START", script)

In [31]:
script_test[:500]

'—_NEWLINE_F:_START This is the story of how I died.<br/>Don’t worry, this is actually a very fun story and the truth is, it isn’t even mine.<br/>This is the story of a girl named Rapunzel and it starts with the sun.</p><p>Now, once upon a time, a single drop of sunlight fell from the heavens and from this small drop of sun, grew a magic, golden flower. It had the ability to heal the sick and injured.<br/>Oh, you see that old woman over there? You might want to remember<br/>her. She’s kind of imp'

In [32]:
script_only = script_test.replace("<br/>", ' ').replace("<p>", ' ').replace('</p>', ' ')

In [33]:
script_only[:500]

'—_NEWLINE_F:_START This is the story of how I died. Don’t worry, this is actually a very fun story and the truth is, it isn’t even mine. This is the story of a girl named Rapunzel and it starts with the sun.  Now, once upon a time, a single drop of sunlight fell from the heavens and from this small drop of sun, grew a magic, golden flower. It had the ability to heal the sick and injured. Oh, you see that old woman over there? You might want to remember her. She’s kind of important.  Well, centur'

In [34]:
script_lines = script_only.split('_NEWLINE_')

In [35]:
script_lines = script_lines[1:]

In [36]:
script_lines[:10]

['F:_START This is the story of how I died. Don’t worry, this is actually a very fun story and the truth is, it isn’t even mine. This is the story of a girl named Rapunzel and it starts with the sun.  Now, once upon a time, a single drop of sunlight fell from the heavens and from this small drop of sun, grew a magic, golden flower. It had the ability to heal the sick and injured. Oh, you see that old woman over there? You might want to remember her. She’s kind of important.  Well, centuries passed, and a hop skip and a boat ride away there grew a kingdom. The kingdom was ruled by a beloved King and Queen. And the Queen, (well,) she was about to have a baby. She got sick. Really, sick. She was running out of time, and that’s when people usually start to look for a miracle. Or in this case, a magic golden flower.  Ah, I told you she’d be important. You see, instead of sharing the sun’s gift, this woman, Mother Gothel, hoarded its healing power and used it to keep herself young for hundre

In [37]:
script_line_tuple = []
for line in script_lines:
    script_line_tuple.append(tuple(line.split(":_START")))

In [38]:
script_line_tuple[:10]

[('F',
  ' This is the story of how I died. Don’t worry, this is actually a very fun story and the truth is, it isn’t even mine. This is the story of a girl named Rapunzel and it starts with the sun.  Now, once upon a time, a single drop of sunlight fell from the heavens and from this small drop of sun, grew a magic, golden flower. It had the ability to heal the sick and injured. Oh, you see that old woman over there? You might want to remember her. She’s kind of important.  Well, centuries passed, and a hop skip and a boat ride away there grew a kingdom. The kingdom was ruled by a beloved King and Queen. And the Queen, (well,) she was about to have a baby. She got sick. Really, sick. She was running out of time, and that’s when people usually start to look for a miracle. Or in this case, a magic golden flower.  Ah, I told you she’d be important. You see, instead of sharing the sun’s gift, this woman, Mother Gothel, hoarded its healing power and used it to keep herself young for hundre

In [39]:
import pandas as pd

In [40]:
tangled = pd.DataFrame(script_line_tuple, columns=["Speaker", "Text"])

In [41]:
tangled.head()

,Speaker,Text
0,F,"This is the story of how I died. Don’t worry,..."
1,G,"Flower, gleam and glow Let your power shine M..."
2,F,"All right, you get the gist. She sings to it,..."
3,Guards,We’ve found it!
4,F,The magic of the golden flower healed the que...


### Awesome!
Just a couple more things to do: Replace abbreviations with full names of characters, lower/strip all the text, and add labels found in the other Disney Data.

In [42]:
tangled.Speaker = tangled.Speaker.map(lambda x: x.strip())

In [43]:
tangled.Text = tangled.Text.map(lambda x: x.strip().lower())

In [44]:
tangled.Text[0] #sweet! It worked!!

'this is the story of how i died. don’t worry, this is actually a very fun story and the truth is, it isn’t even mine. this is the story of a girl named rapunzel and it starts with the sun.  now, once upon a time, a single drop of sunlight fell from the heavens and from this small drop of sun, grew a magic, golden flower. it had the ability to heal the sick and injured. oh, you see that old woman over there? you might want to remember her. she’s kind of important.  well, centuries passed, and a hop skip and a boat ride away there grew a kingdom. the kingdom was ruled by a beloved king and queen. and the queen, (well,) she was about to have a baby. she got sick. really, sick. she was running out of time, and that’s when people usually start to look for a miracle. or in this case, a magic golden flower.  ah, i told you she’d be important. you see, instead of sharing the sun’s gift, this woman, mother gothel, hoarded its healing power and used it to keep herself young for hundreds of year

In [45]:
tangled.Speaker.unique()

array(['F', 'G', 'Guards', 'Young R', 'R', 'St', 'Guard', 'Capt', 'Thug',
       'Vlad', 'Hook', 'Attila', 'Thugs', 'Big', 'Thug Chorus', 'All',
       'Greno', 'Shorty', 'Girl', 'Citizen'], dtype=object)

In [46]:
#replacing character names
char_names = {'F':'flynn', 'Young R': 'young rapunzel', 'R': 'rapunzel', 'St': 'stabbington brothers', 'G': 'gothel'}


In [47]:
char_names['F']

'flynn'

In [48]:
for item in tangled.itertuples():
    if item[1] in char_names.keys():
        tangled.Speaker.iloc[item[0]] = char_names[item[1]]
    else:
        tangled.Speaker.iloc[item[0]] = tangled.Speaker.iloc[item[0]].lower()

In [49]:
tangled.head(10)

,Speaker,Text
0,flynn,"this is the story of how i died. don’t worry, ..."
1,gothel,"flower, gleam and glow let your power shine ma..."
2,flynn,"all right, you get the gist. she sings to it, ..."
3,guards,we’ve found it!
4,flynn,the magic of the golden flower healed the quee...
5,gothel,"flower, gleam and glow let your power shine ma..."
6,flynn,"gothel broke into the castle, stole the child ..."
7,young rapunzel,(heal what has been hurt) (change the fate’s d...
8,flynn,"gothel had found her new magic flower, but thi..."
9,young rapunzel,why can’t i go outside?


# We GUCCHI!!!!
Let's save this dude as a pickle!

In [50]:
tangled.to_pickle(r'..\..\Animated-Movie-Gendered-Dialogue\private\tangled_lines.pkl')